In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
import csv 
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler


In [2]:
df = pd.read_csv("rc_dataset_2000.csv")
#print(df)
print(df.shape)

(2000, 6)


In [3]:


X = df[["R", "C", "Vin"]].values
Y = df[["V_out", "V_R", "V_C"]].values

scaler_x = StandardScaler()
scaler_y = StandardScaler()
X_tensor = torch.tensor(scaler_x.fit_transform(X), dtype = torch.float32)
Y_tensor = torch.tensor(scaler_y.fit_transform(Y), dtype = torch.float32)

print(X_tensor)
print(Y_tensor)

dataset = TensorDataset(X_tensor, Y_tensor)

loader = DataLoader(dataset, batch_size=32, shuffle=True)

tensor([[-0.8704,  0.1741, -1.6475],
        [-0.8704,  0.1741, -1.4741],
        [-0.8704,  0.1741, -1.3007],
        ...,
        [ 1.5667,  1.5667,  1.3007],
        [ 1.5667,  1.5667,  1.4741],
        [ 1.5667,  1.5667,  1.6475]])
tensor([[-1.6475, -0.1321, -1.6475],
        [-1.4741, -0.1321, -1.4741],
        [-1.3007, -0.1321, -1.3007],
        ...,
        [ 1.3007, -0.1321,  1.3007],
        [ 1.4741,  3.6499,  1.4741],
        [ 1.6475,  3.6499,  1.6475]])


In [4]:
class RegressionModel(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.fc1 = nn.Linear(in_features, 3)
        self.fc2 = nn.Linear(3, 6)
        self.fc3 = nn.Linear(6, 9)
        self.fc4 = nn.Linear(9, 12)
        self.fc5 = nn.Linear(12, 12)
        self.fc6 = nn.Linear(12, 9)
        self.fc7 = nn.Linear(9, 6)
        self.fc8 = nn.Linear(6, 3)
        


    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))
        x = self.fc8(x)  # No activation on output for regression
        return x

In [5]:
in_features = 3
out_features = 3
model = RegressionModel(in_features, out_features)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)


In [6]:
nb_epochs = 50

for i in range(nb_epochs):
    for X_batch, Y_batch in loader:
        model.train()
        outputs = model(X_batch)
        loss = criterion(outputs, Y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{i+ 1}], Loss: {loss.item():.9f}')

Epoch [1], Loss: 0.927008629
Epoch [2], Loss: 0.405386925
Epoch [3], Loss: 0.773813188
Epoch [4], Loss: 0.157925740
Epoch [5], Loss: 0.103821538
Epoch [6], Loss: 0.422946662
Epoch [7], Loss: 0.666316569
Epoch [8], Loss: 0.270196408
Epoch [9], Loss: 0.318131357
Epoch [10], Loss: 0.166092798
Epoch [11], Loss: 0.003387291
Epoch [12], Loss: 0.102326907
Epoch [13], Loss: 0.384280831
Epoch [14], Loss: 0.318375707
Epoch [15], Loss: 0.404432178
Epoch [16], Loss: 0.245208308
Epoch [17], Loss: 0.252480417
Epoch [18], Loss: 0.352237582
Epoch [19], Loss: 0.366103560
Epoch [20], Loss: 0.618422091
Epoch [21], Loss: 0.039476778
Epoch [22], Loss: 0.176535785
Epoch [23], Loss: 0.460493326
Epoch [24], Loss: 0.753884256
Epoch [25], Loss: 0.176457599
Epoch [26], Loss: 0.473192602
Epoch [27], Loss: 0.726606369
Epoch [28], Loss: 0.192514598
Epoch [29], Loss: 0.757202208
Epoch [30], Loss: 0.210084006
Epoch [31], Loss: 0.041676681
Epoch [32], Loss: 0.154307872
Epoch [33], Loss: 0.433687776
Epoch [34], Loss: 0

In [7]:
model.eval()

# No gradient calculation during evaluation
with torch.no_grad():
    # Forward pass
    predictions = model(X_tensor)  # Assuming X_tensor is your input data

    # Calculate the loss (optional)
    loss = criterion(predictions, Y_tensor)
    print(f"Evaluation Loss: {loss.item():.3f}")

    predictions_original = scaler_y.inverse_transform(predictions.numpy())

    # Print the inverse transformed predictions
    print("Inverse Predictions: \n", predictions_original)

Evaluation Loss: 0.314
Inverse Predictions: 
 [[1.4955597e-01 2.9376697e-17 1.4734964e-01]
 [6.2461263e-01 2.4522011e-17 6.2271386e-01]
 [1.0996687e+00 1.9667324e-17 1.0980778e+00]
 ...
 [9.2499132e+00 3.8512960e-16 9.2525530e+00]
 [9.2503443e+00 3.8528532e-16 9.2529840e+00]
 [9.2501678e+00 3.8522153e-16 9.2528076e+00]]
